In [15]:
import string
from nltk.corpus import stopwords
import random
import re
import numpy as np
import pandas as pd
import nltk
import os
import nltk.corpus
from nltk import TweetTokenizer
from nltk.tokenize.stanford import StanfordTokenizer
from nltk.util import bigrams, trigrams, ngrams
nltk.download('punkt')
nltk.download('stopwords')
#from langdetect import detect
#import sent2vec
cachedStopWords = stopwords.words("english")
CLEANR = re.compile('<.*?>')


[nltk_data] Downloading package punkt to C:\Users\AJAY
[nltk_data]     BISWAS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\AJAY
[nltk_data]     BISWAS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
def remove_punctuation(text, punct_list):
    for punc in punct_list:
        if punc in text:
            text = text.replace(punc, ' ')
    return text.strip()


def remove_html_tags(raw_html):
    cleantext = re.sub(CLEANR, '', raw_html)
    return cleantext


def tokenize(sentence, to_lower=True, tknzr=TweetTokenizer()):
    """Arguments:
        - tknzr: a tokenizer implementing the NLTK tokenizer interface
        - sentence: a string to be tokenized
        - to_lower: lowercasing or not
    """
    sentence = sentence.strip()
    sentence = ' '.join([format_token(x) for x in tknzr.tokenize(sentence)])
    if to_lower:
        sentence = sentence.lower()
    # replace urls by <url>
    sentence = re.sub(
        '((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', sentence)
    # replace @user268 by <user>
    sentence = re.sub('(\@[^\s]+)', '', sentence)

    filter(lambda word: ' ' not in word, sentence)

    #remove single letter words
    sentence = ' '.join([w for w in sentence.split() if len(w) > 1])

    sentence = remove_html_tags(sentence)
    sentence = remove_punctuation(sentence, list(string.punctuation))
    sentence = ' '.join([word for word in sentence.split()
                        if word not in cachedStopWords])
    return sentence


def format_token(token):
    """"""
    if token == '-LRB-':
        token = '('
    elif token == '-RRB-':
        token = ')'
    elif token == '-RSB-':
        token = ']'
    elif token == '-LSB-':
        token = '['
    elif token == '-LCB-':
        token = '{'
    elif token == '-RCB-':
        token = '}'
    return token


In [17]:
# read comments into lines
df_seed = pd.read_csv("train_set.csv")
df_corpus = pd.read_csv("unlabeled_corpus.csv")

lines_seed = df_seed["comment"]
lines_un = df_corpus["Comment"]

labels_seed = df_seed["label"].apply(str)


In [18]:
# Remove punctuations and stopwords by applying filteredString to each line
# not required
lines_seed = list(map(tokenize, lines_seed))
lines_un = list(map(tokenize, lines_un))

In [19]:
# save as txt
with open('model.txt', mode='wt', encoding='utf-8') as myfile:
    myfile.write('\n'.join(lines_seed))

with open('corpus.txt', mode='wt', encoding='utf-8') as myfile:
    myfile.write('\n'.join(lines_un))


In [20]:
# save as txt with label
lines_seed_sup = ['__label__'.join(x) for x in zip(lines_seed, labels_seed)]
with open('model_supervised.txt', mode='wt', encoding='utf-8') as myfile:
    myfile.write('\n'.join(lines_seed_sup))


In [21]:
# check if number of lines in txt files matches with csv files
no_model = sum(1 for line in open('model.txt'))
no_corpus = sum(1 for line in open('corpus.txt'))
print(no_model,",",len(lines_seed))
print(no_corpus,",",len(lines_un))

376 , 376
14519 , 14519
